In [1]:
import findspark
findspark.init('/home/daniel/spark-3.2.1-bin-hadoop3.2')
from pyspark.sql import SparkSession

In [2]:
spark = SparkSession.builder.appName("Exercise").getOrCreate()

22/04/09 07:01:43 WARN Utils: Your hostname, ubuntu resolves to a loopback address: 127.0.1.1; using 192.168.234.128 instead (on interface ens33)
22/04/09 07:01:43 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
22/04/09 07:01:43 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
22/04/09 07:01:44 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
22/04/09 07:01:44 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.


In [3]:
data = spark.read.csv("cruise_ship_info.csv", header=True, inferSchema=True)

In [11]:
data.printSchema()

root
 |-- Ship_name: string (nullable = true)
 |-- Cruise_line: string (nullable = true)
 |-- Age: integer (nullable = true)
 |-- Tonnage: double (nullable = true)
 |-- passengers: double (nullable = true)
 |-- length: double (nullable = true)
 |-- cabins: double (nullable = true)
 |-- passenger_density: double (nullable = true)
 |-- crew: double (nullable = true)



In [6]:
data.columns

['Ship_name',
 'Cruise_line',
 'Age',
 'Tonnage',
 'passengers',
 'length',
 'cabins',
 'passenger_density',
 'crew']

In [8]:
for item in data.head(1)[0]:
    print(item)

Journey
Azamara
6
30.276999999999997
6.94
5.94
3.55
42.64
3.55


In [12]:
data.select("crew").show()

+----+
|crew|
+----+
|3.55|
|3.55|
| 6.7|
|19.1|
|10.0|
| 9.2|
| 9.2|
| 9.2|
| 9.2|
|11.5|
|11.6|
| 6.6|
| 9.2|
| 9.2|
| 9.3|
|11.6|
|10.3|
| 9.2|
| 9.3|
| 9.2|
+----+
only showing top 20 rows



In [13]:
for ship in data.head(5):
    print(ship, "\n")

Row(Ship_name='Journey', Cruise_line='Azamara', Age=6, Tonnage=30.276999999999997, passengers=6.94, length=5.94, cabins=3.55, passenger_density=42.64, crew=3.55) 

Row(Ship_name='Quest', Cruise_line='Azamara', Age=6, Tonnage=30.276999999999997, passengers=6.94, length=5.94, cabins=3.55, passenger_density=42.64, crew=3.55) 

Row(Ship_name='Celebration', Cruise_line='Carnival', Age=26, Tonnage=47.262, passengers=14.86, length=7.22, cabins=7.43, passenger_density=31.8, crew=6.7) 

Row(Ship_name='Conquest', Cruise_line='Carnival', Age=11, Tonnage=110.0, passengers=29.74, length=9.53, cabins=14.88, passenger_density=36.99, crew=19.1) 

Row(Ship_name='Destiny', Cruise_line='Carnival', Age=17, Tonnage=101.353, passengers=26.42, length=8.92, cabins=13.21, passenger_density=38.36, crew=10.0) 



In [17]:
data.groupBy("Cruise_line").count().orderBy("count").show()

+-----------------+-----+
|      Cruise_line|count|
+-----------------+-----+
|           Orient|    1|
|           Disney|    2|
|          Crystal|    2|
|          Azamara|    2|
|           Cunard|    3|
|         Seabourn|    3|
|         Windstar|    3|
|          Oceania|    3|
|        Silversea|    4|
|Regent_Seven_Seas|    5|
|             Star|    6|
|              P&O|    6|
|              MSC|    8|
|        Celebrity|   10|
|            Costa|   11|
|        Norwegian|   13|
| Holland_American|   14|
|         Princess|   17|
|         Carnival|   22|
|  Royal_Caribbean|   23|
+-----------------+-----+



In [18]:
from pyspark.ml.feature import StringIndexer

In [19]:
indexer = StringIndexer(
    inputCol="Cruise_line",
    outputCol="categoryIndexer"
)

In [20]:
indexed = indexer.fit(data).transform(data)
indexed.show()

+-----------+-----------+---+------------------+----------+------+------+-----------------+----+---------------+
|  Ship_name|Cruise_line|Age|           Tonnage|passengers|length|cabins|passenger_density|crew|categoryIndexer|
+-----------+-----------+---+------------------+----------+------+------+-----------------+----+---------------+
|    Journey|    Azamara|  6|30.276999999999997|      6.94|  5.94|  3.55|            42.64|3.55|           16.0|
|      Quest|    Azamara|  6|30.276999999999997|      6.94|  5.94|  3.55|            42.64|3.55|           16.0|
|Celebration|   Carnival| 26|            47.262|     14.86|  7.22|  7.43|             31.8| 6.7|            1.0|
|   Conquest|   Carnival| 11|             110.0|     29.74|  9.53| 14.88|            36.99|19.1|            1.0|
|    Destiny|   Carnival| 17|           101.353|     26.42|  8.92| 13.21|            38.36|10.0|            1.0|
|    Ecstasy|   Carnival| 22|            70.367|     20.52|  8.55|  10.2|            34.29| 9.2|

In [24]:
indexed.groupBy("categoryIndexer").count().orderBy("categoryIndexer").show()

+---------------+-----+
|categoryIndexer|count|
+---------------+-----+
|            0.0|   23|
|            1.0|   22|
|            2.0|   17|
|            3.0|   14|
|            4.0|   13|
|            5.0|   11|
|            6.0|   10|
|            7.0|    8|
|            8.0|    6|
|            9.0|    6|
|           10.0|    5|
|           11.0|    4|
|           12.0|    3|
|           13.0|    3|
|           14.0|    3|
|           15.0|    3|
|           16.0|    2|
|           17.0|    2|
|           18.0|    2|
|           19.0|    1|
+---------------+-----+



In [25]:
from pyspark.ml.linalg import Vectors
from pyspark.ml.feature import VectorAssembler

In [26]:
indexed.columns

['Ship_name',
 'Cruise_line',
 'Age',
 'Tonnage',
 'passengers',
 'length',
 'cabins',
 'passenger_density',
 'crew',
 'categoryIndexer']

In [27]:
assembler = VectorAssembler(
    inputCols=[
        'Age',
        'Tonnage',
        'passengers',
        'length',
        'cabins',
        'passenger_density',
        'categoryIndexer'
    ],
    outputCol="features"
)

In [28]:
output = assembler.transform(indexed)

In [31]:
output.head(1)

[Row(Ship_name='Journey', Cruise_line='Azamara', Age=6, Tonnage=30.276999999999997, passengers=6.94, length=5.94, cabins=3.55, passenger_density=42.64, crew=3.55, categoryIndexer=16.0, features=DenseVector([6.0, 30.277, 6.94, 5.94, 3.55, 42.64, 16.0]))]

In [29]:
final_data = output.select("features", "crew")

In [32]:
train_data, test_data = final_data.randomSplit([0.7, 0.3])

In [33]:
train_data.describe().show()

+-------+------------------+
|summary|              crew|
+-------+------------------+
|  count|               110|
|   mean| 7.857636363636375|
| stddev|3.2250908321673304|
|    min|              0.59|
|    max|              13.6|
+-------+------------------+



In [34]:
test_data.describe().show()

+-------+-----------------+
|summary|             crew|
+-------+-----------------+
|  count|               48|
|   mean|7.648750000000003|
| stddev|4.104728785712317|
|    min|             1.46|
|    max|             21.0|
+-------+-----------------+



In [36]:
from pyspark.ml.regression import LinearRegression

In [37]:
lr = LinearRegression(labelCol="crew")

In [38]:
model = lr.fit(train_data)

22/04/09 07:17:41 WARN Instrumentation: [a1efbb73] regParam is zero, which might cause numerical instability and overfitting.
22/04/09 07:17:41 WARN InstanceBuilder$NativeBLAS: Failed to load implementation from:dev.ludovic.netlib.blas.JNIBLAS
22/04/09 07:17:41 WARN InstanceBuilder$NativeBLAS: Failed to load implementation from:dev.ludovic.netlib.blas.ForeignLinkerBLAS
22/04/09 07:17:41 WARN InstanceBuilder$NativeLAPACK: Failed to load implementation from:dev.ludovic.netlib.lapack.JNILAPACK


In [39]:
results = model.evaluate(test_data)

In [41]:
results.rootMeanSquaredError

1.2222093926068405

In [42]:
results.r2

0.9094547485155073

In [44]:
results.meanAbsoluteError

0.6529915298644158

In [43]:
train_data.describe().show()

+-------+------------------+
|summary|              crew|
+-------+------------------+
|  count|               110|
|   mean| 7.857636363636375|
| stddev|3.2250908321673304|
|    min|              0.59|
|    max|              13.6|
+-------+------------------+



In [45]:
from pyspark.sql.functions import corr

In [48]:
data.select(corr("crew", "cabins")).show()

+------------------+
|corr(crew, cabins)|
+------------------+
|0.9508226063578497|
+------------------+

